**Netzwerke - Hands-On**

---
## Gliederung:
- Setup
- Netzwerkerstellung
- Bereinigung und *largest connected component*
- Maßzahlen
- Grafische Darstellung
- Communities

**Netzwerke - Hands-On**

---
## Imports und Setup

In [ ]:
import networkx as nx
import sqlalchemy as sa
from tqdm.notebook import tqdm

import src
from src.bokeh_graph import BokehGraph
from src.connect import create_sqlite_session
from src.models import Author

In [ ]:
engine, s = create_sqlite_session(src.PATH / "data/example.db")

**Netzwerke - Hands-On**

---
## Netzwerk Konstruktion

In [ ]:
# Query alle Autoren
authors = s.query(Author).options(sa.orm.subqueryload(Author.items))
authors[0]

Author(pk_authors=507, fullname=Dewitte, S, lastname=Dewitte, firstname=Siegfried, middlename=None, author_group=None, role=author, orcid_id=None, orcid_id_tr=None, r_id=None, r_id_tr=None)

**Netzwerke - Hands-On**

---
Funktion `add_node`:
- nimmt `author` Objekte und ein `Graph` Objekt
- fügt Autor\*innen dem Netzwerk hinzu

In [ ]:
def add_node(author, net):
    node = author.pk_authors

    if not graph.has_node(node):
        graph.add_node(
            node,
            firstname=author.firstname,
            lastname=author.lastname,
            n_items=len(author.items),
        )
        
    return node

**Netzwerke - Hands-On**

---
- initialisiert ein `graph` Objekt
- für jedes `author` Objekt und dessen `coauthor` Objekte wird die `add_node` Funktion durchgeführt

In [ ]:
graph = nx.Graph()

for author in tqdm(authors, total=authors.count()):
    # add node if not exists
    node = add_node(author, graph)

    # add coauthors and edges that do not exist yet
    for coauthor in author.coauthors:
        co_node = add_node(coauthor, graph)

        if not graph.has_edge(node, co_node):
            graph.add_edge(node, co_node)

  0%|          | 0/16546 [00:00<?, ?it/s]

**Netzwerke - Hands-On**

---
## Bereinigung:
- Isolierte *nodes* entfernen
- *large connected component* auswählen
- Eckdaten prüfen

In [ ]:
# Identifikation der isolates
isolates = nx.isolates(graph)

# Alle nodes die sich in 'isolates' befinden werden entfernt*
nodes = list(graph.nodes)
for node in nodes:
    if node in isolates:
        graph.remove_node(node)

# _____________________________________________________________
# in diesem Netzwerk befinden sich allerdings keine isolates

In [ ]:
# Subnetzwerke absteigend nach Anzahl der nodes sortieren
largest_components = sorted(nx.connected_components(graph), key=len, reverse=True)

# subnetzwerke mit der Funktion 'subgraph' zuweisen
lcc = graph.subgraph(largest_components[0])
subgraph = graph.subgraph(largest_components[1])

In [ ]:
# Einige Eckdaten des Gesamtnetzwerkes, des größten und zweitgrößten component

print('                 | Gesamt | LCC    | 2ndLCC')
print('Anzahl der nodes | ', len(graph.nodes()), '|', len(lcc.nodes), '  |', len(subgraph))
print('Anzahl der edges | ', graph.size(), '|', lcc.size(), ' |', subgraph.size())
print('Dichte           |',
      round(nx.density(graph), 4), '|',
      round(nx.density(lcc), 4), '|',
      round(nx.density(subgraph), 4), '\n')

                 | Gesamt | LCC    | 2ndLCC
Anzahl der nodes |  16546 | 8986   | 64
Anzahl der edges |  82280 | 65714  | 212
Dichte           | 0.0006 | 0.0016 | 0.1052 



In [ ]:
degree = nx.degree(subgraph)
cc = nx.closeness_centrality(subgraph)
bc = nx.betweenness_centrality(subgraph)
ec = nx.eigenvector_centrality(subgraph)

degree = dict(degree)
nx.set_node_attributes(subgraph, degree, "degree")
nx.set_node_attributes(subgraph, name='closeness', values=cc)
nx.set_node_attributes(subgraph, name='betweenness', values=bc)
nx.set_node_attributes(subgraph, name='eigenvector', values=ec)

**Netzwerke - Hands-On**

---
Grafische Darstellung

In [ ]:
from bokeh.io import output_file

(src.PATH / "tmp").mkdir(exist_ok=True)
output_file(src.PATH / "tmp/myoutput.html")

In [ ]:
plot = BokehGraph(subgraph, width=1920, height=1080, inline=False)

import copy
plot.layout()
lay = copy.deepcopy(plot._layout)

In [ ]:
plot.draw(
    color_by="degree", palette="viridis",
    edge_alpha=0.5, max_colors=6, node_size=25
)

![](images/lcc_graph.png)

**Netzwerke - Hands-On**

---
## "Echte" communities

In [ ]:
from networkx.algorithms import community

communities = community.naive_greedy_modularity_communities(subgraph)

In [ ]:
len(communities)

6

In [ ]:
com_dict = {}
com_number = 0
for com in tqdm(communities):
    com_number += 1
    for node in com:
        com_dict[node] = com_number

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
nx.set_node_attributes(subgraph, name='community', values=com_dict)

In [ ]:
plot = BokehGraph(subgraph, width=1920, height=1080, inline=False)
plot._layout = lay
plot.draw(
    color_by="community", palette="viridis",
    edge_alpha=0.5, max_colors=6, node_size=25
)

**Netzwerke - Caveat Emptor**

---
- Unvollständige Daten
    - was passiert wenn wir 1,2,...10...100 nodes zufällig entfernen?
    - Dichte, Zentralität, Triaden, Communities, ...
    
    
- Netzwerkdefinition und Netzwerkgrenzen
    - Simple Fragen mit möglicherweise komplexen Konsequenzen:
        - Wie definieren wir Verbindungen?
        - Können wir ein Netzwerk als abgeschlossen betrachten?
        
        
- Wie macht sich das in Koautorschaftsnetzwerken bemerkbar?
    - Namens- und Institutionendisambiguierung
    - Vollständigkeit der Koautorenlisten
    - Welche Netzwerke haben wir letztendlich?

**Netzwerke - Abschluss**

---
#### Was nützt uns das?
- Netzwerkpositionen und Parameter können z.B. anhängige und unabhängige Variablen sein:
    - Hängt Zentralität (auch) von Zitationen ab?
    - Wie sehr überlappen sich inhaltliche Profile von Autor\*innen mit *communities*
    - Können bestehende *communities* zukünftige inhaltliche Profile bestimmen?
    - Wie wirkt sich über die Zeit steigende/sinkende Zentralität auf Erfolg aus?

**Netzwerke - Abschluss**

---
## Ausblick:
- ergm / tergm
- siena

**Exkurs - ERGM**

---
## Exponential Random Graph Models


**Idee**
- Untersuchung ob edges zwischen zwei nodes existieren oder nicht (vgl. logistische Regression)
- Das empirische Netzwerk ist eines unter vielen möglichen Netzwerken (mit der gleichen Anzahl an nodes und edges)

**Exkurs - ERGM**

---
**Vorgehen**
1. Bindungen sind Zufallsvariablen
2. Es bestehen möglicherweise Abhängigkeitshypothesen (Homophilie, Triadische Schließung, ...)
3. Punkt 2 macht spezifische Modellformen erforderlich
4. Vereinfachung von Modellparametern (z.B. durchschnittliche triadische Schließung)
5. Modellschätzung

&rarr; Wie wahrscheinlich ist es, das empirische Netzwerk unter den gegebenen Bedingungen zu beobachten?

**Exkurs - ERGM**

---
### Modellierung

- bisher nicht in Python möglich
- R package [`ergm`](https://cran.r-project.org/web/packages/ergm/ergm.pdf) (`tergm` für Modellierungen über Zeit, Teil von `statnet`)
- es ist möglich diverse Abhängigkeitsparameter aufzunehmen

**Exkurs - SIENA**

---
## SIENA*, ein Stochastic Actor Oriented Model (SAOM)

**Idee**
- Untersuchung der Entscheidung (gerichtete) Verbindungen mit Alteri einzugehen
- &rarr; Fokus auf den Handlungen von Akteuren

**Besonderheiten**
- Auf längsschnittliche Daten ausgelegt
- Die Auflösung von edges kann modelliert werden

---
\* Simulation Investigation for Empirical Network Analysis

**Exkurs - SIENA**

---

### Modellierung
- ebenfalls nicht mit Python modellierbar
- R package [`RSiena`](http://www.stats.ox.ac.uk/~snijders/siena/RSiena_Manual.pdf)

**Fazit Modellierung**

---
#### "Es kommt drauf an"
- ergm &rarr; Bindungszentriert
- SIENA &rarr; Akteurszentriert


**Abschluss**

---
#### Was nützt uns das?
- Netzwerkpositionen und Parameter können z.B. anhängige und unabhängige Variablen sein:
    - Hängt Zentralität (auch) von Zitationen ab?
    - Wie sehr überlappen sich inhaltliche Profile von Autor\*innen mit *communities*
    - Können bestehende *communities* zukünftige inhaltliche Profile bestimmen?
    - Wie wirkt sich über die Zeit steigende/sinkende Zentralität auf Erfolg aus?